In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [45]:
import matplotlib.pyplot as plt
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchvision import transforms

In [46]:
train_data_raw=pd.read_csv('./data/MNIST/mnist_train.csv').astype('float32')
test_data_raw=pd.read_csv('./data/MNIST/mnist_test.csv').astype('float32')

In [47]:
train_label=train_data_raw['label'].values
train_img=train_data_raw.drop('label', axis=1)

In [48]:
null_bool=train_img.isnull().to_numpy()
a=np.ones((784,1))
nullz=np.sum(np.dot(null_bool,a))
if nullz:
    print('there is missing data')
else:
    print('no missing data')

no missing data


In [49]:
train_img=train_img.values/255

test_img=test_data_raw.values/255
all_img=np.concatenate((train_img, test_img), axis=0)

train_img=train_img.reshape(-1,1,28,28)
test_img=test_img.reshape(-1,1,28,28)
all_img=all_img.reshape(-1,1,28,28)
print(all_img.shape)    


(70000, 1, 28, 28)


In [53]:
transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.RandomResizedCrop(28,scale=(0.85, 1.0)),
                        transforms.RandomAffine(15, translate=(0.15,0.15))
                        
                       ])

In [54]:
# train_img, val_img, train_label, val_label = train_test_split(train_img, train_label, test_size=0.1, random_state=42)

In [60]:
tf_img=np.zeros((train_img.shape))
print(tf_img[0].shape)
print(transform(train_img))
for i in range(train_img.shape[0]):
    tf_img[i]=transform(train_img[i])


(1, 28, 28)


ValueError: pic should be 2/3 dimensional. Got 4 dimensions.

In [59]:
train_imgs=np.vstack((train_img,tf_img))
train_lab=np.hstack((train_label,train_label))
print(train_imgs.shape, train_lab.shape)

(84000, 1, 28, 28) (84000,)


In [61]:
print(train_imgs.shape, train_lab.shape, test_img.shape)

(84000, 1, 28, 28) (84000,) (28000, 1, 28, 28)


In [ ]:
# train_img, val_img, train_label, val_label = train_test_split(train_imgs, train_lab, test_size=0.1, random_state=42)

In [ ]:
train_img, train_label=T.from_numpy(train_imgs).type(T.FloatTensor), T.from_numpy(train_lab).type(T.LongTensor)
# train_img, train_label=T.from_numpy(train_img).type(T.FloatTensor), T.from_numpy(train_label).type(T.LongTensor)
# val_img, val_label=T.from_numpy(val_img).type(T.FloatTensor), T.from_numpy(val_label).type(T.LongTensor)
test_img=T.from_numpy(test_img).type(T.FloatTensor)

# print(val_img.size(), val_label.size(), train_img.shape, train_label.shape)

In [ ]:
print(train_img.shape, train_label.shape, test_img.shape)
# print(train_img.shape, train_label.shape, val_img.shape, val_label.shape,test_img.shape)

In [ ]:
fig = plt.figure(figsize=(6,3))
fig, axs = plt.subplots(4,3)
fig.suptitle('Vertically stacked subplots')
for i in range(2):
    for j in range(3):
        axs[i,j].imshow(train_img[(i*3+j)*8,:][0], cmap='gray')
for i in range(2,4):
    for j in range(3):
        axs[i,j].imshow(tf_img[((i-2)*3+j)*8,:][0], cmap='gray')
plt.show()

class DopeNet(nn.Module):
    
    def __init__(self):
        super(DopeNet, self).__init__()
        
        self.fc1= nn.Linear(784, 128)
        self.dropout= nn.Dropout(0.2)
        self.fc2= nn.Linear(128,96)
        self.fc3= nn.Linear(96, 10)
        
    
    def forward(self, input_x):
        x = F.relu(self.fc1(input_x))
        x = F.relu(self.dropout(self.fc2(x)))
        x = F.relu(self.fc3(x))
        
        return x
    
model=DopeNet().to('cuda')
print(model)

In [ ]:
class DopeNet(nn.Module):
    
    def __init__(self):
        super(DopeNet, self).__init__()
        
        self.conv1= nn.Conv2d(1, 16, 5, stride=1, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2= nn.Conv2d(16, 32, 5, stride=1, padding=2)
        self.bn2 = nn.BatchNorm2d(32)
        self.fc1= nn.Linear(32*7*7,128)
        self.fc2= nn.Linear(128,64)
        self.fc3= nn.Linear(64, 10)
        
    
    def forward(self, input_x):
        x = F.max_pool2d(F.relu(self.bn1(self.conv1(input_x))),2)
        x = F.max_pool2d(F.relu(self.bn2(self.conv2(x))),2)
        x = x.view(-1,32*7*7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model=DopeNet().to('cuda')
print(model)

In [ ]:
###HYPERPARAMETER
n_epoch=30
batch_size=128

train = T.utils.data.TensorDataset(train_img, train_label)
# train = T.utils.data.TensorDataset(train_img, tr_label)
# val = T.utils.data.TensorDataset(val_img, val_label)


In [ ]:
fig = plt.figure(figsize=(6,3))
plt.imshow(train_img[1][0], cmap='gray')
plt.show()
fig = plt.figure(figsize=(6,3))
plt.imshow(tf_img[1][0], cmap='gray')
plt.show()

In [ ]:
#create dataloader
train_loader= T.utils.data.DataLoader(train, batch_size=batch_size, shuffle=False)
# val_loader= T.utils.data.DataLoader(val, batch_size=batch_size, shuffle=False)


In [ ]:
###HYPERPARAMETER
lr=0.0075
betas=(0.9,0.999)
momentum=0.9
decay_rate=0.95

criterion=nn.CrossEntropyLoss()
# optimizer=optim.Adam(model.parameters(), lr=lr, betas=betas)
optimizer=optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# lr_sched = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay_rate)
lr_sched = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [ ]:
#train loop
count=0
loss_list=[]
val_list=[]
iteration=[]

for i in range(n_epoch):
    model.train()
    for j, (img, label) in enumerate(train_loader):
        img = img.to('cuda')
        label = label.to('cuda')
        optimizer.zero_grad()
        output=model(img)
        loss=criterion(output,label)
        loss.backward()
        optimizer.step()
        count+=1    
        
        if count%50 == 0:
            correct=0
            total=0
            model.eval()
            for images, labels in val_loader:
                with T.no_grad():
                    images=images.to('cuda')
                    labels=labels.to('cuda')
                    outputs=model(images)
                    val_loss=criterion(outputs,labels)
                    pred=T.max(outputs.data, 1)[1]
                    total+=len(labels)
#                 print(len(images), len(labels), len(pred))
                    correct += (pred == labels).sum()
            accuracy = 100.0 * float(correct) / float(total)
            loss_list.append(loss.item())
            val_list.append(val_loss.item())
            iteration.append(count)
    lr_sched.step()
            

    print('Epoch {} - Training Loss: {} | Val Loss:{} |  Val Accuracy:{}'.format(i, loss.item(), val_loss.item(),accuracy))
#     print('Epoch {} - Training Loss: {} | Val Loss: |  Val Accuracy:'.format(i, loss.item()))

In [ ]:
data=test_img[5][0]
plt.imshow(data, cmap='gray')
test_data=T.tensor(data).view(-1,1,28,28)

with T.no_grad():
    test_data=test_img.to('cuda')
    logps = model(test_data)

    
ps = T.exp(logps)
ps=ps.cpu()
probab = list(ps.numpy()[0])
print("Predicted Digit =", probab.index(max(probab)))

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(iteration,loss_list)
# plt.plot(iteration,val_list)
plt.xlabel("Number of iteration")
plt.ylabel("Loss")
plt.title("Logistic Regression: Loss vs Number of iteration")
plt.show()

In [ ]:
T.save(model.state_dict(), 'mnist_inf_mj.pt')

In [ ]:
# T.save(model, 'mnist_mj_mod.pt')

In [ ]:
test = T.utils.data.TensorDataset(test_img)
print(test_img.shape)
test_loader= T.utils.data.DataLoader(test, batch_size=1, shuffle=False)

In [ ]:
predicts=[]
model.eval()
for [images] in test_loader:
    with T.no_grad():
        images=images.to('cuda')
        outputs=model(images)
        pred=T.max(outputs.data, 1)[1]
        predicts.append(pred.item())
        

In [ ]:
print(len(predicts))


In [ ]:
id = range(1,28001)
print(id[3])

In [ ]:
df = pd.DataFrame(list(zip(id,predicts)), columns=['ImageId', 'Label'])

In [ ]:
df.to_csv('mj_mnist_cnn_6.csv', index=False)